# Importacoes

In [224]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Iniciando o chrome com uma porta de depuracao

In [225]:
opts = Options()
opts.add_experimental_option('debuggerAddress', 'localhost:9250')
service = webdriver.ChromeService(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=service, options=opts)
navegador.get('https://www.upwork.com/nx/find-work/best-matches')

# Buscando por palavra chave

In [226]:
for tentativa in range(2):
    try:
        search_box = WebDriverWait(navegador, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[aria-labelledby="cfe-job-search-label"]'))
        )

        search_box.click()  # Clica no campo
        search_box.clear()  # Limpa o campo, se necessário
        search_box.send_keys('data engeneer')  # Envia a busca
        search_box.send_keys(Keys.RETURN)  # Envia a busca
        break  # Sai do loop se a operação for bem-sucedida
    except Exception as e:
        print(f"Erro na tentativa {tentativa + 1}: {e}")
        if tentativa == 1:  # Se for a segunda tentativa e ainda houver erro
            print("Falha ao tentar buscar após 2 tentativas.")

Erro na tentativa 1: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00F8D933+25811]
	(No symbol) [0x00F1E314]
	(No symbol) [0x00E12523]
	(No symbol) [0x00E180A4]
	(No symbol) [0x00E19988]
	(No symbol) [0x00E19A00]
	(No symbol) [0x00E58ED2]
	(No symbol) [0x00E4EFD2]
	(No symbol) [0x00E7AD2C]
	(No symbol) [0x00E4D475]
	(No symbol) [0x00E7AFC4]
	(No symbol) [0x00E946F0]
	(No symbol) [0x00E7AAC6]
	(No symbol) [0x00E4BEFD]
	(No symbol) [0x00E4C8FD]
	GetHandleVerifier [0x0125F143+2981091]
	GetHandleVerifier [0x012B2FF9+3324825]
	GetHandleVerifier [0x0101B32F+605903]
	GetHandleVerifier [0x01022CBC+637020]
	(No symbol) [0x00F26F4D]
	(No symbol) [0x00F23DD8]
	(No symbol) [0x00F23F75]
	(No symbol) [0x00F16406]
	BaseThreadInitThunk [0x753C7B

# Extraindo de mais de uma pagina

In [227]:
dicionarios_vagas = []

# Defina um número máximo de páginas a serem processadas (opcional)
max_paginas = 5
pagina_atual = 0

while pagina_atual < max_paginas:
    # Obter todos os artigos na página atual
    artigos = navegador.find_elements(By.TAG_NAME, 'article')

    for artigo in artigos:
        texto = artigo.text
        array_resultado = texto.split('\n')

        # Verifica se o primeiro índice é 'FEATURED' e ajusta os índices se necessário
        if array_resultado[0] == 'FEATURED':
            # Incrementa todos os índices que serão utilizados
            dicionario_vaga = {
                'tempo_postado': array_resultado[1],  # índice 1
                'titulo': array_resultado[2],          # índice 2
                'pagamento_verificado': array_resultado[5],  # índice 5
                'rating_cliente': array_resultado[7],      # índice 7
                'valor_gasto': array_resultado[8],         # índice 8
                'localizacao': array_resultado[10],        # índice 10
                'tipo_pagamento': array_resultado[11],     # índice 11
                'nivel_dificuldade': array_resultado[12],  # índice 12
                'estimativa_projeto': array_resultado[14], # índice 14
                'descricao_vaga': array_resultado[15],     # índice 15
                'palavras_chaves': array_resultado[16:]    # índice 16 em diante
            }
        else:
            # Não precisa ajustar os índices
            dicionario_vaga = {
                'tempo_postado': array_resultado[0],  # índice 0
                'titulo': array_resultado[1],          # índice 1
                'pagamento_verificado': array_resultado[4],  # índice 4
                'rating_cliente': array_resultado[6],      # índice 6
                'valor_gasto': array_resultado[7],         # índice 7
                'localizacao': array_resultado[9],        # índice 9
                'tipo_pagamento': array_resultado[10],     # índice 10
                'nivel_dificuldade': array_resultado[11],  # índice 11
                'estimativa_projeto': array_resultado[13], # índice 13
                'descricao_vaga': array_resultado[14],     # índice 14
                'palavras_chaves': array_resultado[15:]     # índice 15 em diante
            }

        # Adicionar o dicionário à lista
        dicionarios_vagas.append(dicionario_vaga)

    # Tentar clicar no botão "Next"
    try:
        button_next = WebDriverWait(navegador, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'air3-pagination-next-btn'))
        )
        button_next.click()  # Clicar no botão "Next"
        pagina_atual += 1  # Incrementar o contador de páginas

        time.sleep(1)
    except Exception as e:
        print(f"Erro ao clicar no botão 'Next': {e}")
        break  # Sair do loop se não conseguir clicar no botão "Next"

In [228]:
len(dicionarios_vagas)

50

# criando dataframe

In [229]:
df_vagas = pd.DataFrame(dicionarios_vagas)

In [223]:
caminho_arquivo = 'arquivos/data engineer.xlsx'
df_vagas.to_excel(caminho_arquivo, index=False)